# LP

In [1]:
import pulp
import pandas as pd
import os

In [2]:
model_BESS = pulp.LpProblem('Minimize_cost', pulp.LpMinimize)

# Params

T = 24  # Time
Efficiency = 93/100    # efficiency of BESS
B = 750    # BESS capacity
W = 250    # Max power of BESS
P_base = 0    # Basefee


# Sets

PERIOD = list(range(0,T))
P_tou = [40.3, 40.3, 40.3, 40.3, 40.3, 40.3, 40.3, 40.3, 40.3, 85.0, 150.9, 150.9, 85.0, 150.9, 150.9, 150.9, 150.9, 85.0, 85.0, 85.0, 85.0, 85.0, 85.0, 40.3]
Load = pd.read_csv(os.path.join("result", 'LOAD_RESULT.csv'), usecols=['Total Load'])
PV = pd.read_csv(os.path.join("result", 'PV_RESULT.csv'), usecols=['pv'])
L = Load['Total Load']
S = PV['pv']

# Variables

u = pulp.LpVariable.dicts('BESS State of Charge', [t for t in PERIOD], lowBound=0.2, upBound=0.8, cat='Continuous')
d_load = pulp.LpVariable.dicts('Diesel to Load at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
d_bess = pulp.LpVariable.dicts('Diesel to BESS at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
v_load = pulp.LpVariable.dicts('PV to Load at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
v_bess = pulp.LpVariable.dicts('PV to BESS at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
c = pulp.LpVariable.dicts('BESS to Load at t', [t for t in PERIOD], lowBound=0, cat='Continuous')

In [5]:
import orloge as ol

# Constraints

# Initial and Final SoC
model_BESS += u[0] == 0.5
model_BESS += u[T-1] == 0.5


# Load Constraint
for t in PERIOD:
    model_BESS += d_load[t] + v_load[t] + Efficiency * c[t] == L[t]


# PV Constraint
for t in PERIOD:
    model_BESS += v_bess[t] + v_load[t] == S[t]


# SoC Constraint
for t in range(0,T-2):
    model_BESS += u[t] + (Efficiency * (d_bess[t] + v_bess[t]) - c[t]) / B == u[t+1]


# BESS power Constraint
for t in PERIOD:
    model_BESS += c[t] + Efficiency * (d_bess[t] + v_bess[t]) <= W


# Objective Function
for t in PERIOD:
        model_BESS.objective += pulp.lpSum(P_tou[t] * (d_bess[t]+d_load[t]))


path = "log_file.txt"
solver = pulp.PULP_CBC_CMD(msg=False, timeLimit=99, logPath=path) # We set msg=False since logPath disables CBC's logs in the Python console
# Model Solve
model_BESS.solve(solver)

logs_dict = ol.get_info_solver(path, 'CBC') # Orloge returns a dict with all logs info
best_bound, best_solution = logs_dict["best_bound"], logs_dict["best_solution"]

In [4]:
charge = []
discharge = []

for t in PERIOD:
    charge.append((d_bess[t].varValue+v_bess[t].varValue)*Efficiency)
    discharge.append(c[t].varValue)

charge


result = {'PERIOD' : PERIOD,
        'CHARGE' : charge,
        'DISCHARGE' : discharge,
        'Obj' : pulp.value(model_BESS.objective)}

result = pd.DataFrame(result)


df_new = result[['PERIOD', 'CHARGE', 'DISCHARGE']].copy()
df_new.loc[0, 'Obj'] = result['Obj'].iloc[0]

df_new.to_csv('BESS 최적 스케쥴링 결과값.csv', index=False)
